# Initialization

In [75]:
############ INITIALIZATION #############

'''IMPORTS'''
from __future__ import print_function
import facebook
import json
import networkx as nx
from networkx.algorithms import bipartite
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import urllib
import requests
'''JASON DUMP FUNCTION'''
def pp(o): 
    print(json.dumps(o, indent=1))
'''OPEN CONNECTION'''
g = facebook.GraphAPI('EAACEdEose0cBANuG9DxVZBhOlCsmi1P4ehYchOOyQ8qXrmwnAXkgOmhrETTDu6gwMFEt6Pnn0zp0IC3ikhu2AbSUJN3WMChUascMD5OB9ZBusFMLJWu6tEojKDcY0ZAQJYTBfUu6s5ZCEY8oEgGXAg2ZAyf40NSTBgKf7y12gZCc5pzJgZBRoUhaWZAftKtcRZCnnZCvfkTA12AgZDZD',version='2.2')
#pp(g.get_object('me')) #TEST CONNECTION

In [116]:
############ SEARCH PAGES ############
'''REQUESTS'''
republicanSearch = g.request("search", {'q' : 'Republican National Committee', 'type' : 'page'})  
print(republicanSearch)
democratSearch = g.request("search", {'q' : 'Democratic Party', 'type' : 'page'})

{'paging': {'next': 'https://graph.facebook.com/v2.4/search?access_token=EAACEdEose0cBANuG9DxVZBhOlCsmi1P4ehYchOOyQ8qXrmwnAXkgOmhrETTDu6gwMFEt6Pnn0zp0IC3ikhu2AbSUJN3WMChUascMD5OB9ZBusFMLJWu6tEojKDcY0ZAQJYTBfUu6s5ZCEY8oEgGXAg2ZAyf40NSTBgKf7y12gZCc5pzJgZBRoUhaWZAftKtcRZCnnZCvfkTA12AgZDZD&q=Republican+National+Committee&type=page&limit=25&after=MjQZD', 'cursors': {'after': 'MjQZD', 'before': 'MAZDZD'}}, 'data': [{'id': '123192635089', 'name': 'Republican National Committee'}, {'id': '6507308380', 'name': 'College Republican National Committee'}, {'id': '46093654473', 'name': 'National Republican Congressional Committee'}, {'id': '1663030593997939', 'name': 'Republican National Committee'}, {'id': '953103421391825', 'name': "Young Professionals Committee of the Women's National Republican Club"}, {'id': '534066910017503', 'name': 'Toni Anne Dashiell, Republican National Committeewoman for Texas'}, {'id': '1819112328325457', 'name': 'National Republican Senatorial Committee'}, {'id': '16303

In [77]:
########### OBTAIN PAGE IDs #############
#These will be used later to query the pages for information 

#RNC is 123192635089

'''CODE'''
rncID = ''
dncID = ''
for data in republicanSearch['data']:
    if data['name'] == 'Republican National Committee':
        rncID = data['id']

for data in democratSearch['data']:
    if data['name'] == 'Democratic Party':
        dncID = data['id']

'''OUTPUT'''
print('RNC ID:',rncID)
print('DNC ID:',dncID)

RNC ID: 1630358130514449
DNC ID: 12301006942


# Finding Nodes/Edges
## Functions

In [122]:
##### FUNCTIONS ######
def postIDs(pageID, iterations=1):
    '''Return list of post ids.'''
    ids = []
    pagePosts = g.get_object(id=pageID, fields='posts')
    for i in range(iterations):
        try:
            pp(pagePosts)
            for post in pagePosts['posts']['data']:
                ids.append(post['id'])
            try:
                pageNext = pagePosts['posts']['paging']['next']
                pagePosts = requests.get(pagePosts['posts']['paging']['next']).json()
            except:
                break
        except:
            for post in pagePosts['data']:
                ids.append(post['id'])
            try:
                pageNext = pagePosts['paging']['next']
                pagePosts = requests.get(pagePosts['paging']['next']).json()
            except:
                break
    return ids

def buildComments(postID, graph, iterations=1):
    '''Return new graph with edges between posts and comments.'''
    new_graph = graph.copy()
    print("Checking",postID)
    comments = g.get_object(postID, fields='comments')
    for i in range(iterations):
        print("Iteration",i+1)
        try:
            for comment in comments['comments']['data']:
                commentData = {}
                commentData['id'] = comment['from']['id']
                new_graph.add_node(commentData['id'],attr_dict={'id':commentData['id'],'bipartite':1, 'color':'blue'})
                new_graph.add_edge(postID,commentData['id'])
            try:
                pageNext = comments['comments']['paging']['next']
                comments = requests.get(comments['comments']['paging']['next']).json()
            except:
                print('Comment Chain Ended')
                break
        except:
            for comment in comments['data']:
                commentData = {}
                commentData['id'] = comment['from']['id']
                new_graph.add_node(commentData['id'],attr_dict={'id':commentData['id'],'bipartite':1, 'color':'blue'})
                new_graph.add_edge(postID,commentData['id'])
            try:
                pageNext = comments['paging']['next']
                comments = requests.get(comments['paging']['next']).json()
            except:
                print('Comment Chain Ended')
                break
                
    return new_graph

def buildPostNodes(graph, ids):
    new_graph = graph.copy()
    for id in ids:
        postLikes = g.get_connections(id,'likes', summary=True)['summary']['total_count']
        new_graph.add_node(id,attr_dict={'name':id, 'likes':postLikes, 'bipartite':0, 'color':'red'})
    return new_graph

In [123]:
### BEGIN GRAPHS ###
dncGraph = nx.Graph()
rncGraph = nx.Graph()

In [124]:
### OBTAIN POST NODES ###
'''REQUESTS'''
print('begin')
rncPostIDs = postIDs('123192635089',4)
print('next')
dncPostIDs = postIDs(dncID,4)

print('next')
rncPostGraph = buildPostNodes(rncGraph,rncPostIDs)
print('next')
dncPostGraph = buildPostNodes(dncGraph,dncPostIDs)

begin
{
 "id": "123192635089",
 "posts": {
  "paging": {
   "previous": "https://graph.facebook.com/v2.4/123192635089/posts?since=1506901801&access_token=EAACEdEose0cBANuG9DxVZBhOlCsmi1P4ehYchOOyQ8qXrmwnAXkgOmhrETTDu6gwMFEt6Pnn0zp0IC3ikhu2AbSUJN3WMChUascMD5OB9ZBusFMLJWu6tEojKDcY0ZAQJYTBfUu6s5ZCEY8oEgGXAg2ZAyf40NSTBgKf7y12gZCc5pzJgZBRoUhaWZAftKtcRZCnnZCvfkTA12AgZDZD&limit=25&__paging_token=enc_AdCAyrOKKgtn2nBUqBBLZBRGd3uOs4i28kpZCPocHZAZAoKGRZCnkM1oPJZCRVONc8htxIKuQYIFBFvH4gSpJIK1Slnl2OLQ8Q1UIEqyHHLRv6fmH05QZDZD&__previous=1",
   "next": "https://graph.facebook.com/v2.4/123192635089/posts?access_token=EAACEdEose0cBANuG9DxVZBhOlCsmi1P4ehYchOOyQ8qXrmwnAXkgOmhrETTDu6gwMFEt6Pnn0zp0IC3ikhu2AbSUJN3WMChUascMD5OB9ZBusFMLJWu6tEojKDcY0ZAQJYTBfUu6s5ZCEY8oEgGXAg2ZAyf40NSTBgKf7y12gZCc5pzJgZBRoUhaWZAftKtcRZCnnZCvfkTA12AgZDZD&limit=25&until=1506437401&__paging_token=enc_AdAGsYN0aDQBsQ3nPATZAxJUBdaFaZCFAw5GIifxZA2QhtymYHZAJl7U2sfCcJxpounzhNeftt0KwuEMZBCXfE0hjtyZCIiTCDnhw9cwc48WMDGBmZBBQZDZD"
  },
  "da

In [125]:
### Obtain Comments and Draw Edges ###
rncGraphComments = rncPostGraph.copy()
dncGraphComments = dncPostGraph.copy()
for id in rncPostIDs:
    rncGraphComments = buildComments(id,rncGraphComments,6)
print('Moving On')
for id in dncPostIDs:
    dncGraphComments = buildComments(id,dncGraphComments,6)


Checking 123192635089_10155493746090090
Iteration 1
Comment Chain Ended
Checking 123192635089_10155493536680090
Iteration 1
Iteration 2
Iteration 3
Comment Chain Ended
Checking 123192635089_10155493161870090
Iteration 1
Iteration 2
Comment Chain Ended
Checking 123192635089_10155492616115090
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Comment Chain Ended
Checking 123192635089_10155491251585090
Iteration 1
Iteration 2
Comment Chain Ended
Checking 123192635089_10155490995020090
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Checking 123192635089_10155490520700090
Iteration 1
Iteration 2
Comment Chain Ended
Checking 123192635089_10155488474860090
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Comment Chain Ended
Checking 123192635089_10155488240645090
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Checking 123192635089_10155487772445090
Iteration 1
Iteration 2
Comment Chain Ended
Checking

In [132]:
print('DNC Before:',len(dncGraphComments.nodes()))
print('RNC Before:',len(rncGraphComments.nodes()))
dncGraphFiltered = dncGraphComments.copy()
rncGraphFiltered = rncGraphComments.copy()
dncBipartiteOne = []
dncBipartiteZero = []
rncBipartiteOne = []
rncBipartiteZero = []

for node in dncGraphComments.nodes():
    if len(dncGraphComments.edges(node)) < 5:
        dncGraphFiltered.remove_node(node)
for node in rncGraphComments.nodes():
    if len(rncGraphComments.edges(node)) < 5:
        rncGraphFiltered.remove_node(node)
        
for node in dncGraphFiltered.nodes():
    print(dncGraphFiltered.node[node])
    if dncGraphFiltered.node[node]['attr_dict']['bipartite'] == 1:
        dncBipartiteOne.append(node)
    elif dncGraphFiltered.node[node]['attr_dict']['bipartite'] == 0:
        dncBipartiteZero.append(node)
        
for node in rncGraphFiltered.nodes():
    if rncGraphFiltered.node[node]['attr_dict']['bipartite'] == 1:
        rncBipartiteOne.append(node)
    elif rncGraphFiltered.node[node]['attr_dict']['bipartite'] == 0:
        rncBipartiteZero.append(node)
        
print('DNC After:',len(dncGraphFiltered.nodes()))
print('RNC After:',len(rncGraphFiltered.nodes()))
print('RNC Bipartite 0:',len(rncBipartiteZero))
print('RNC Bipartite 1:',len(rncBipartiteOne))
print('DNC Bipartite 0:',len(dncBipartiteZero))
print('DNC Bipartite 1:',len(dncBipartiteOne))

DNC Before: 5430
RNC Before: 5708
{'attr_dict': {'id': '10202520524634805', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '840735562614340', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '476809002420423', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '10202718643831134', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '1463344833957535', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'bipartite': 0, 'likes': 1466, 'color': 'red', 'name': '12301006942_10155228040846943'}}
{'attr_dict': {'id': '794650710585825', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '1548332608514773', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '362136353935944', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '583467421783754', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '10102730511335350', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'id': '10201991167459246', 'bipartite': 1, 'color': 'blue'}}
{'attr_dict': {'bipartite': 0, 'likes'

In [29]:
dncZero = bipartite.projected_graph(dncGraphFiltered, dncBipartiteZero)
dncOne = bipartite.projected_graph(dncGraphFiltered, dncBipartiteOne)
rncZero = bipartite.projected_graph(rncGraphFiltered, rncBipartiteZero)
rncOne = bipartite.projected_graph(rncGraphFiltered, rncBipartiteOne)

In [30]:
nx.write_graphml(dncGraphFiltered,"dncGraphFiltered.graphml")
nx.write_graphml(rncGraphFiltered,'rncGraphFiltered.graphml')